In [7]:
import xgboost as xgb 
import pandas as pd 
import numpy as np
import os
from tqdm import tqdm

In [8]:
bid_model = xgb.XGBClassifier()
bid_model.load_model("../model_weight/bid_using.model")
ask_model = xgb.XGBClassifier()
ask_model.load_model("../model_weight/ask_using.model")

In [11]:
PATH = r"../stock_dataset/2330"
DATE = "20221230"

FILE_PATH = os.path.join(PATH,DATE+".csv")
data = pd.read_csv(FILE_PATH)


HOLD_CAPITAL = 10000
HOLD_STOCK = 0

# buy = 0 , sell = 1 ,nothing = -1
predict_action = -1 
want_price = -1
tick_gap =0.5

loop = tqdm(range(len(data)))
for idx in loop :
    
    tick = data.iloc[idx].to_numpy()
    
    cur_match_price = tick[2]

    if cur_match_price == want_price:

        # buy
        if predict_action == 0 :

            HOLD_STOCK = HOLD_CAPITAL/cur_match_price
            HOLD_CAPITAL = 0
        # sell 
        elif predict_action == 1:
            HOLD_CAPITAL = HOLD_STOCK*cur_match_price
            HOLD_STOCK = 0 


    input_data = [[tick[3]]]    
    input_data[0].extend(tick[11:16])
    input_data[0].extend(tick[21:26])

    if HOLD_CAPITAL == 0:
        output = bid_model.predict_proba(input_data)
        choose = np.argmax(output[0])
        
        if choose == 1 :
            predict_action= 1
            want_price = cur_match_price+ tick_gap
        else :
            want_price = -1
            predict_action = -1

    elif HOLD_STOCK == 0:
        output = ask_model.predict_proba(input_data)
        choose = np.argmax(output[0])

        if choose == 1 :
            predict_action= 0
            want_price = cur_match_price -  tick_gap
        else :
            want_price = -1 
            predict_action = -1

    loop.set_description(f"[{idx}/{len(data)}] {DATE} , capital: {HOLD_CAPITAL} , stock: {HOLD_STOCK}")
    loop.update
    if idx == len(data)-1:
        final_profit =HOLD_CAPITAL+HOLD_STOCK*cur_match_price
        print(f"final profit: {final_profit} , ROI: {final_profit/10000:2%}")
    




[5360/5361] 20221230 , capital: 0 , stock: 58.85747359498651: 100%|██████████| 5361/5361 [04:05<00:00, 21.84it/s] 

final profit: 26574.14932813641 , ROI: 265.741493%
